In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv('train_test.csv')

In [2]:
import nltk
MY_STOP_WORDS = ['u','dont']
def normalize(inp):
    wnl = nltk.stem.wordnet.WordNetLemmatizer()
    stop_words = set(nltk.corpus.stopwords.words('english') + MY_STOP_WORDS)
    tokenizer = nltk.tokenize.RegexpTokenizer('\w+')

    tokens = tokenizer.tokenize(inp)
    tokens = [token.lower() for token in tokens]

    filtered = []
    for token,pos in nltk.pos_tag(tokens):
        if token not in stop_words:
            pos = pos[0].lower()
            if pos in ['a','n','v']:
                filtered.append(wnl.lemmatize(token,pos))
            else:
                filtered.append(wnl.lemmatize(token))
    return filtered
class word_counter:
    def __init__(self):
        self.dic = {}
    def add_spam(self,msg):
        for word in msg:
            if word in self.dic:
                self.dic[word][1] += 1
            else:
                self.dic[word] = [0,1]
    def add_ham(self,msg):
        for word in msg:
            if word in self.dic:
                self.dic[word][0] += 1
            else:
                self.dic[word] = [1,0]

In [3]:
_hams  = data[data.type ==  'ham']
_spams = data[data.type ==  'spam']
hams_list  = _hams.apply(lambda x: normalize(x['text']), axis=1)
spams_list = _spams.apply(lambda x: normalize(x['text']), axis=1)
hams_set   = hams_list.apply(lambda x: set(x))
spams_set  = spams_list.apply(lambda x: set(x))

In [5]:
list_counter = word_counter()
set_counter  = word_counter()
hams_list.apply(lambda x: list_counter.add_ham(x))
spams_list.apply(lambda x: list_counter.add_spam(x))
hams_set.apply(lambda x: set_counter.add_ham(x))
spams_set.apply(lambda x: set_counter.add_spam(x))
list_counter.dic

{'fight': [11, 0],
 'world': [33, 2],
 'easy': [17, 12],
 'either': [12, 10],
 'win': [14, 60],
 'lose': [35, 3],
 'bt': [19, 14],
 'fightng': [3, 0],
 'some1': [7, 0],
 'close': [16, 3],
 'dificult': [3, 0],
 'still': [138, 5],
 'ì_': [114, 0],
 'come': [305, 5],
 'check': [42, 2],
 'nuerologist': [1, 0],
 'lolnice': [1, 0],
 'go': [481, 30],
 'fish': [3, 0],
 'water': [10, 0],
 'wait': [96, 17],
 'e': [78, 11],
 'car': [38, 0],
 'dat': [38, 0],
 'bore': [17, 4],
 'wat': [102, 1],
 'co': [57, 49],
 'outside': [15, 0],
 'get': [544, 91],
 'nothing': [29, 3],
 '2': [309, 187],
 'home': [151, 2],
 'stuff': [42, 0],
 'watch': [61, 0],
 'tv': [20, 8],
 'maybe': [27, 2],
 'westshore': [1, 0],
 'hyde': [2, 0],
 'park': [12, 1],
 'village': [3, 2],
 'place': [52, 5],
 'near': [9, 0],
 'house': [40, 1],
 'know': [238, 25],
 'anthony': [2, 0],
 'bring': [33, 4],
 'money': [49, 4],
 'school': [27, 0],
 'fee': [3, 0],
 'pay': [38, 2],
 'rent': [11, 0],
 'like': [210, 12],
 'thats': [38, 0],
 'nee

In [6]:
set_counter.dic

{'fightng': [3, 0],
 'easy': [17, 12],
 'some1': [5, 0],
 'lose': [26, 3],
 'win': [10, 58],
 'dificult': [3, 0],
 'either': [12, 10],
 'bt': [17, 14],
 'world': [31, 2],
 'fight': [10, 0],
 'close': [16, 3],
 'still': [135, 5],
 'ì_': [91, 0],
 'come': [282, 5],
 'nuerologist': [1, 0],
 'check': [42, 2],
 'lolnice': [1, 0],
 'water': [10, 0],
 'fish': [3, 0],
 'go': [431, 30],
 'e': [70, 8],
 'watch': [59, 0],
 'bore': [16, 4],
 'dat': [37, 0],
 'wait': [93, 17],
 'outside': [15, 0],
 'nothing': [29, 3],
 'tv': [20, 8],
 'car': [38, 0],
 'co': [56, 47],
 '2': [254, 143],
 'wat': [97, 1],
 'stuff': [40, 0],
 'home': [149, 2],
 'get': [496, 88],
 'village': [3, 1],
 'hyde': [2, 0],
 'westshore': [1, 0],
 'maybe': [26, 2],
 'park': [12, 1],
 'near': [8, 0],
 'house': [38, 1],
 'place': [50, 5],
 'money': [46, 3],
 'like': [197, 12],
 'bring': [30, 4],
 'school': [24, 0],
 'friend': [73, 15],
 'fee': [3, 0],
 'know': [222, 21],
 'need': [153, 10],
 'rent': [10, 0],
 'pay': [32, 2],
 'help